In [54]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import json
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableBranch, RunnableLambda
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
from langchain.tools.retriever import create_retriever_tool
from typing import List, Union
from langchain_community.tools import Tool
import os

In [3]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [4]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# llm = OllamaFunctions(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", format="json", temperature=0)
eeve = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)
qwen2 = ChatOllama(model="qwen2:latest", temperature=0)

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_NEWS_INDEX = "MY_NEWS_INDEX"
vectorstore1 = FAISS.load_local(MY_NEWS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출

/Users/user/Library/Caches/pypoetry/virtualenvs/test-langchain-lw2NDlv9-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/user/Library/Caches/pypoetry/virtualenvs/test-langchain-lw2NDlv9-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="saved_news_search",
    description="""
다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:
- 엔비디아의 스타트업 인수 관련 내용
- 퍼플렉시티 관련 내용 (회사가치, 투자 등)
- 라마3 관련 내용
""",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="""
다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:
- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용
- 생성 AI 규제 연구 관련 내용
- 생성 AI 연구 관련 내용
"""
)

tools = [retriever_tool1, retriever_tool2]
tools

[Tool(name='saved_news_search', description='\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x118b59620>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x119eae590>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x118b59800>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x119eae590>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description=

In [57]:
# prompt_for_extract_actions = ChatPromptTemplate.from_messages([
#     ("system", """
# 당신은 인간의 질문에 답변하기 위해 적절한 도구를 선택하는 AI 어시스턴트입니다. 

# 다음 도구들을 사용할 수 있습니다:
# {tools}

# 인간의 질문을 주의 깊게 분석하고, 가장 적절한 도구를 선택하여 답변하세요. 질문에 따라 여러 도구를 사용해야 할 수도 있습니다.

# 응답 시 다음 JSON 형식을 엄격히 따라주세요:
# ```json
# [
#   {{
#     "action": string, // 선택한 도구의 이름 (tool_name)
#     "action_input": string // 도구에 입력할 검색어 또는 질문
#   }},
#   {{
#     // 다음 액션 정보
#   }}
# ]
# ```

# 응답 지침:
# 1. 항상 JSON 배열로 응답하세요, 단일 도구를 사용하는 경우에도 마찬가지입니다.
# 2. 하나의 도구만 필요한 경우, 배열에 하나의 객체만 포함시키세요.
# 3. 여러 도구가 필요한 경우, 각 도구에 대해 별도의 객체를 배열에 추가하세요.
# 4. 액션의 순서가 중요한 경우, 배열 내 객체의 순서로 표현하세요.
# 5. 이 JSON 형식으로만 응답하고, 다른 설명이나 추가 텍스트는 포함하지 마세요.
# 6. 인간의 질문에 직접 답변하지 말고, 적절한 도구를 선택하여 JSON 형식으로만 응답하세요.
# 7. 적절한 도구를 찾지 못하거나 도구 사용이 필요하지 않다고 판단되는 경우, "action"을 "None"으로, "action_input"을 빈 문자열로 설정하여 응답하세요.

# question: {question}

# answer: 
# """
#     )
# ])
prompt_for_extract_actions = hub.pull("kwonempty/extract-actions-for-ollama")

def get_tools(query):
    """
    사용 가능한 도구들의 이름과 설명을 JSON 형식으로 반환
    """
    # tools 리스트에서 각 도구의 이름, 설명을 딕셔너리 형태로 추출
    tool_info = [{"tool_name": tool.name, "tool_description": tool.description} for tool in tools]
    
    print(f"get_tools / tool_info: {tool_info}")
    
    # tool_info 리스트를 JSON 형식으로 변환하여 반환
    return json.dumps(tool_info, ensure_ascii=False)

chain_for_extract_actions = (
    {"tools": get_tools, "question": RunnablePassthrough()}
    | prompt_for_extract_actions 
    | qwen2
    | StrOutputParser()
    )

In [71]:
chain_for_extract_actions.invoke("3+4 계산해줘")

get_tools / tool_info: [{'tool_name': 'saved_news_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n'}, {'tool_name': 'pdf_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용\n- 생성 AI 규제 연구 관련 내용\n- 생성 AI 연구 관련 내용\n'}]


'[\n  {\n    "action": "None",\n    "action_input": ""\n  }\n]'

In [11]:
# query = "라마3 성능은?"
# query = "생성형 AI 도입에 따른 규제 연구 책임자는?"
query = "라마3 성능은 어떻게 돼? 그리고 생성형 AI 도입에 따른 규제 연구 책임자는 누구야?"
actions_json = chain_for_extract_actions.invoke(query)
actions_json

get_tools / tool_info: [{'tool_name': 'saved_news_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n'}, {'tool_name': 'pdf_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용\n- 생성 AI 규제 연구 관련 내용\n- 생성 AI 연구 관련 내용\n'}]


'[\n  {\n    "action": "saved_news_search",\n    "action_input": "라마3 성능"\n  },\n  {\n    "action": "pdf_search",\n    "action_input": "생성형 AI 도입에 따른 선거 규제 연구 책임자"\n  }\n]'

In [66]:
def get_documents_from_actions(actions_json: str, tools: List[Tool]) -> List[Document]:
    """
    주어진 JSON 문자열을 파싱하여 해당 액션에 대응하는 검색기를 찾아서 
    액션을 실행 후 검색된 문서를 반환
    
    :param actions_json: 액션과 그 입력이 포함된 JSON 문자열
    :param tools: 사용 가능한 도구들의 리스트
    :return: 액션을 통해 검색된 문서들의 리스트
    """
    print(f"get_documents_from_actions / actions_json: {actions_json}")
    
    # JSON 문자열을 파싱
    try:
        actions = json.loads(actions_json)
    except json.JSONDecodeError:
        raise ValueError("유효하지 않은 JSON 문자열")

    # 파싱된 객체가 리스트인지 확인
    if not isinstance(actions, list):
        raise ValueError("제공된 JSON은 액션 리스트를 나타내야 함")

    documents = []

    # 도구 이름으로 검색기를 가져오는 함수
    def get_retriever_by_tool_name(name: str) -> VectorStoreRetriever:
        for tool in tools:
            if tool.name == name:
                return tool.func.keywords['retriever']
        return None

    # 각 액션을 처리
    for action in actions:
        if not isinstance(action, dict) or 'action' not in action or 'action_input' not in action:
            continue  # 유효하지 않은 액션은 건너뜀

        tool_name = action['action']
        action_input = action['action_input']
        print(f"get_documents_from_actions / tool_name: {tool_name} / action_input: {action_input}")
        
        if tool_name == "None": # 사용할 도구 없음. 바로 빈 document 리턴
            print(f"get_documents_from_actions / 사용할 도구 없음. 바로 빈 document 리턴")
            return []
        
        retriever = get_retriever_by_tool_name(tool_name)
        
        if retriever:
            # 액션 입력으로 검색기 실행
            retrieved_docs = retriever.invoke(action_input)
            documents.extend(retrieved_docs)
        
    print(f"get_documents_from_actions / len(documents): {len(documents)}")
    return documents

In [67]:
get_documents_from_actions(actions_json, tools)

get_documents_from_actions / actions_json: [
  {
    "action": "saved_news_search",
    "action_input": "라마3 성능"
  },
  {
    "action": "pdf_search",
    "action_input": "생성형 AI 도입에 따른 선거 규제 연구 책임자"
  }
]
get_documents_from_actions / tool_name: saved_news_search / action_input: 라마3 성능
get_documents_from_actions / tool_name: pdf_search / action_input: 생성형 AI 도입에 따른 선거 규제 연구 책임자
get_documents_from_actions / len(documents): 6


[Document(page_content="라마 3 벤치마크 결과 (사진=메타)\n\n\n라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에는 강하지만, 70B의 경우 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 떨어졌다.\xa0\n특히 인간 선호도에서 경쟁 모델을 앞서는 것으로 알려졌다.\n조언 요청, 브레인스토밍, 분류, 비공개 질문 답변, 코딩, 창의적인 글쓰기, 추출, 공개 질문 답변, 추론, 재작성 및 요약 등 12가지 주요 사용 사례를 포함한 1800개\xa0프롬프트\xa0구축\xa0데이터셋에 대한 인간 평가에서 오픈AI의 'GPT-3.5', 미스트랄 7B, 클로드 3 소네트보다 높게 평가됐다.\n\n\n라마 3 인간 평가 결과 (사진=메타)", metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='라마 3 인간 평가 결과 (사진=메타)\n\n\n허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드\xa01위에 오르며 역대 가장 빠른 1위 달성 기록을 세웠다.\n또 이전 라마 1과 2를 기반으로 3만개 이상의 새로운 모델이 출시됐으며, 라마 2 모델은 1700억번 다운로드됐다는 통계치도 공개해 눈길을 모았다.\xa0\n다만 라마 3는 완전한 오픈 소스가 아니다.\xa0연구용 및 상업용으로 모두 사용할 수 있지만, 개발자가 다른 생성 모델을 훈련하기 위해 모델을 사용하는 것을 금지한다.\n\n\n메타 AI (사진=메타)', metadata={'source': 'https://www.aitimes.com/news/articleView.html?idxno=158943'}),
 Document(page_content='특히 15조개 이상의 토큰을 동원, 학습량이 라마 2 대비 7배 이상 많으며 코드량은 4배 더 많다. 다만 데이터셋은 공

In [68]:
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 정확하고 신뢰할 수 있는 답변을 제공하는 유능한 업무 보조자야.
아래의 context를 사용해서 question에 대한 답변을 작성해줘.

다음 지침을 따라주세요:
1. 답변은 반드시 한국어로 작성해야 해.
2. context에 있는 정보만을 사용해서 답변해야 해.
3. 정답을 확실히 알 수 없다면 "주어진 정보로는 답변하기 어렵습니다."라고만 말해.
4. 답변 시 추측하거나 개인적인 의견을 추가하지 마.
5. 가능한 간결하고 명확하게 답변해.

# question: 
{question}

# context: 
{context}

# answer:
"""
    ),
])

default_prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 정확하고 신뢰할 수 있는 답변을 제공하는 유능한 업무 보조자야.
다음 질문에 최선을 다해서 대답해줘.

# question: 
{question}

# answer:
"""
    ),
])

retrieved_docs = []
def get_page_contents_with_metadata(docs) -> str: 
    """
    문서 리스트를 받아 각 문서의 본문 내용과 출처를 포함한 문자열을 생성
    """
    global retrieved_docs
    retrieved_docs = docs
    
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc.page_content}\n### 출처: {doc.metadata['source']}"
    return result

def get_retrieved_docs_string(query) -> str:
    """
    쿼리에 따라 문서를 검색하고, 해당 문서들의 본문 내용과 출처를 포함한 문자열을 반환
    """
    actions_json = chain_for_extract_actions.invoke(query)
    docs = get_documents_from_actions(actions_json, tools)
    return get_page_contents_with_metadata(docs)

def get_metadata_sources(docs) -> str: 
    """
    문서 리스트에서 각 문서의 출처 추출해서 문자열로 반환
    """
    sources = set()
    for doc in docs:
        source = doc.metadata['source']
        is_pdf = source.endswith('.pdf')
        if (is_pdf):
            file_path = doc.metadata['source']
            file_name = os.path.basename(file_path)
            source = f"{file_name} ({int(doc.metadata['page']) + 1}페이지)"
        sources.add(source)
    return "\n".join(sources)

def check_context(inputs: dict) -> bool:
    """
    context 존재 여부 확인
    
    :return: 문자열이 비어있지 않으면 True, 비어있으면 False
    """
    result = bool(inputs['context'].strip())
    print(f"check_context / result: {result}")
    return result

def parse(ai_message: AIMessage) -> str:
    """
    AI 메시지 파싱해서 내용에 출처 추가
    """
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

with_context_chain = (
    RunnablePassthrough()
    | RunnableLambda(lambda x: {
        "context": x["context"],
        "question": x["question"]
    })
    | agent_prompt
    | eeve
    | parse
)

without_context_chain = (
    RunnablePassthrough()
    | RunnableLambda(lambda x: {"question": x["question"]})
    | default_prompt
    | eeve
    | StrOutputParser()
)

agent_chain = (
    {"context": get_retrieved_docs_string, "question": RunnablePassthrough()}
    | RunnableBranch(
        (lambda x: check_context(x), with_context_chain),
        without_context_chain  # default
    )
)

In [69]:
# agent_chain.invoke("퍼플렉시티가 투자받은 금액?") # saved_news_search / '퍼플렉시티는 투자받은 금액이 약 6300만 달러(약 860억 원)이며, 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가받았습니다.'
# agent_chain.invoke("생성형 AI 도입에 따른 규제 연구 책임자는?") # pdf_search / '해당 문서에 따르면, 생성형 AI 도입과 관련된 규제 연구를 담당하는 연구 책임자는 김주희 교수님입니다.'
agent_chain.invoke("라마3 성능은 어떻게 돼? 그리고 생성형 AI 도입에 따른 규제 연구 책임자는 누구야?")

get_tools / tool_info: [{'tool_name': 'saved_news_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n'}, {'tool_name': 'pdf_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용\n- 생성 AI 규제 연구 관련 내용\n- 생성 AI 연구 관련 내용\n'}]
get_documents_from_actions / actions_json: [
  {
    "action": "saved_news_search",
    "action_input": "라마3 성능"
  },
  {
    "action": "pdf_search",
    "action_input": "생성형 AI 도입에 따른 선거 규제 연구 책임자"
  }
]
get_documents_from_actions / tool_name: saved_news_search / action_input: 라마3 성능
get_documents_from_actions / tool_name: pdf_search / action_input: 생성형 AI 도입에 따른 선거 규제 연구 책임자
get_documents_from_actions / len(documents): 6
check_context / result: True


'라마3의 성능은 객관식 문제(MMLU)와 코딩(HumanEval)에서 강점을 보이며, 특히 인간 선호도 측면에서 경쟁 모델을 앞서고 있습니다. 그러나 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 비해 뒤처집니다.\n\n라마3의 규제 연구 책임자는 국립부경대학교 김주희 교수입니다. 이 연구는 생성형 AI 도입과 관련된 다양한 문제에 대비하기 위한 정책적 방향성을 설정하는 것을 목적으로 합니다. 연구 범위는 중범위 메타 분석으로, 2000년대 이후 소셜 미디어(SNS) 도입 및 빅데이터의 선거 활용에 따른 영향력과 문제점을 다룹니다.\n\n[출처]\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (2페이지)\nhttps://www.aitimes.com/news/articleView.html?idxno=158943\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (7페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (1페이지)'

In [70]:
agent_chain.invoke("12+34 계산해줘")

get_tools / tool_info: [{'tool_name': 'saved_news_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 엔비디아의 스타트업 인수 관련 내용\n- 퍼플렉시티 관련 내용 (회사가치, 투자 등)\n- 라마3 관련 내용\n'}, {'tool_name': 'pdf_search', 'tool_description': '\n다음과 같은 정보를 검색할 때에는 이 도구를 사용해야 한다:\n- 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 내용\n- 생성 AI 규제 연구 관련 내용\n- 생성 AI 연구 관련 내용\n'}]
get_documents_from_actions / actions_json: [
  {
    "action": "None",
    "action_input": ""
  }
]
get_documents_from_actions / tool_name: None / action_input: 
get_documents_from_actions / 사용할 도구 없음. 바로 빈 document 리턴
check_context / result: False


'물론이죠, 도와드리겠습니다!\n\n12 + 34 = 46입니다.'